In [47]:
import sqlite3

# connection to a SQLite database in memory
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# SALES table
cursor.execute('''
CREATE TABLE SALES (
    Date TEXT,
    Order_id INTEGER,
    Item_id INTEGER,
    Customer_id INTEGER,
    Quantity INTEGER,
    Revenue REAL
)
''')

# ITEMS table
cursor.execute('''
CREATE TABLE ITEMS (
    Item_id INTEGER,
    Item_name TEXT,
    Price REAL,
    department TEXT
)
''')

# CUSTOMERS table
cursor.execute('''
CREATE TABLE CUSTOMERS (
    Customer_id INTEGER,
    First_name TEXT,
    Last_name TEXT,
    Address TEXT
)
''')

# data into SALES table
cursor.executemany('''
INSERT INTO SALES (Date, Order_id, Item_id, Customer_id, Quantity, Revenue) VALUES (?, ?, ?, ?, ?, ?)
''', [
    ('2023-03-18', 1, 1, 1, 2, 200),
    ('2023-03-18', 2, 2, 2, 1, 100),
    ('2023-01-15', 3, 3, 1, 3, 300)
])

# data into ITEMS table
cursor.executemany('''
INSERT INTO ITEMS (Item_id, Item_name, Price, department) VALUES (?, ?, ?, ?)
''', [
    (1, 'Item A', 100.0, 'Electronics'),
    (2, 'Item B', 100.0, 'Books'),
    (3, 'Item C', 100.0, 'Clothing')
])

# data into CUSTOMERS table
cursor.executemany('''
INSERT INTO CUSTOMERS (Customer_id, First_name, Last_name, Address) VALUES (?, ?, ?, ?)
''', [
    (1, 'John', 'Doe', '123 Elm St'),
    (2, 'Jane', 'Smith', '456 Oak St')
])


conn.commit()

**Executing queries**

In [49]:
# Total number of orders on 18th March 2023
cursor.execute('''
SELECT COUNT(DISTINCT Order_id) AS total_orders
FROM SALES
WHERE Date = '2023-03-18'
''')
total_orders = cursor.fetchone()[0]
print("Total Orders on March 18th:", total_orders)

Total Orders on March 18th: 2


In [51]:
# Total number of John Doe orders on 18th March 2023
cursor.execute('''
SELECT COUNT(DISTINCT s.Order_id) AS john_doe_orders
FROM SALES s
JOIN CUSTOMERS c ON s.Customer_id = c.Customer_id
WHERE s.Date = '2023-03-18'
  AND c.First_name = 'John'
  AND c.Last_name = 'Doe'
''')
john_doe_orders = cursor.fetchone()[0]

print("John Doe Orders on March 18th:", john_doe_orders)

John Doe Orders on March 18th: 1


In [53]:
# Total number of customers who purchased in January 2023 and average spend per customer
cursor.execute('''
SELECT COUNT(DISTINCT Customer_id) AS total_customers, AVG(total_spend) AS avg_spend_per_customer
FROM (
  SELECT Customer_id, SUM(Revenue) AS total_spend
  FROM SALES
  WHERE Date BETWEEN '2023-01-01' AND '2023-01-31'
  GROUP BY Customer_id
) AS customer_totals;
''')
january_customers = cursor.fetchone()
print("January Customers:", january_customers[0], "Average Spend:", january_customers[1])

January Customers: 1 Average Spend: 300.0


In [55]:
# Departments that generated less than $600 in 2022
cursor.execute('''
SELECT i.department, SUM(s.Revenue) AS total_revenue
FROM SALES s
JOIN ITEMS i ON s.Item_id = i.Item_id
WHERE strftime('%Y', s.Date) = '2022'
GROUP BY i.department
HAVING SUM(s.Revenue) < 600;
''')
low_revenue_departments = cursor.fetchall()

print("Low Revenue Departments:", low_revenue_departments)

Low Revenue Departments: []


In [57]:
# Most and least revenue generated by an order
cursor.execute('''
SELECT MAX(Revenue) AS max_revenue, MIN(Revenue) AS min_revenue
FROM SALES;
''')
revenue_range = cursor.fetchone()
print("Revenue Range:", revenue_range)

Revenue Range: (300.0, 100.0)


In [59]:
# Orders from the most lucrative order
cursor.execute('''
SELECT *
FROM SALES
WHERE Revenue = (SELECT MAX(Revenue) FROM SALES);
''')
most_lucrative_order = cursor.fetchall()

print("Most Lucrative Order Details:", most_lucrative_order)


Most Lucrative Order Details: [('2023-01-15', 3, 3, 1, 3, 300.0)]
